In [1]:
import torch
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '6'

import random
import time
import argparse
import copy
import logging

from scipy import io as sio

from scipy import signal
from torch.autograd import Variable
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from archs.cross_baseline_cat import BioPhysNet
from datasets.rppg_datasets_reconstruction import COHFACE, PURE, UBFC, VIPL, MMSEHR
from losses.NPLoss import Neg_Pearson
from losses.CELoss import TorchLossComputer
from utils.utils import AvgrageMeter, cxcorr_align, pearson_correlation_coefficient
import math


def cal_psd_hr(output, Fs, return_type):
    
    cur_device = output.device
    
    def compute_complex_absolute_given_k(output, k, N):
        two_pi_n_over_N = 2 * math.pi * torch.arange(0, N, dtype=torch.float) / N
        hanning = torch.from_numpy(np.hanning(N)).type(torch.FloatTensor).view(1, -1)

        k = k.type(torch.FloatTensor).to(cur_device)
        two_pi_n_over_N = two_pi_n_over_N.to(cur_device)
        hanning = hanning.to(cur_device)
        
        output = output.view(1, -1) * hanning
        output = output.view(1, 1, -1).type(torch.cuda.FloatTensor)
        k = k.view(1, -1, 1)
        two_pi_n_over_N = two_pi_n_over_N.view(1, 1, -1)
        complex_absolute = torch.sum(output * torch.sin(k * two_pi_n_over_N), dim=-1) ** 2 \
                           + torch.sum(output * torch.cos(k * two_pi_n_over_N), dim=-1) ** 2
        return complex_absolute
    
    output = output.view(1, -1)
    
    N = output.size()[1]
    bpm_range = torch.arange(40, 180, dtype=torch.float)
    unit_per_hz = Fs / N
    feasible_bpm = bpm_range / 60.0
    k = feasible_bpm / unit_per_hz
    
    # only calculate feasible PSD range [0.7,4]Hz
    complex_absolute = compute_complex_absolute_given_k(output, k, N)
    complex_absolute = (1.0 / complex_absolute.sum()) * complex_absolute
    complex_absolute = complex_absolute.view(-1)
    whole_max_val, whole_max_idx = complex_absolute.max(0) # max返回（values, indices）
    whole_max_idx = whole_max_idx.type(torch.float) # 功率谱密度的峰值对应频率即为心率

    if return_type == 'psd':
        return complex_absolute
    elif return_type == 'hr':
        return whole_max_idx + 40	# Analogous Softmax operator


def set_seed(seed=92):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def _init_fn(seed=92):
    np.random.seed(seed)

set_seed()

frame_rate = 30
bpm_range = torch.arange(40, 180, dtype=torch.float)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


In [2]:
parser = argparse.ArgumentParser()
## general parameters
parser.add_argument('--num_rppg', type=int, default=160, help='the number of rPPG')
parser.add_argument('--drop_rate', type=int, default=0.2, help='the drop rate of CodeResNet')
parser.add_argument('--train_model', type=str, default='hemnet', help='train_model = [hemnet]')
parser.add_argument('--dataset', type=str, default='PURE')
### add for codephys
parser.add_argument('--batch_size', type=int, default=2)
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--gamma', type=float, default=0.5)
parser.add_argument('--weight_decay', type=float, default=5e-5)
parser.add_argument('--step_size', type=int, default=50)
parser.add_argument('--eval_step', type=int, default=1)
parser.add_argument('--epochs', type=int, default=300)
parser.add_argument('--echo_batches', type=int, default=400, help='the number of mini-batches to print the loss')
parser.add_argument('--save_path', type=str, default="/data/chushuyang/hemnet_exp_results", help='the path to save the model [ckpt, code, visulization]')

parser.add_argument('--align', type=bool, default=False)
parser.add_argument('--proir_weight', default=1e-4)
parser.add_argument('--appearance_weight', default=1e-3)
parser.add_argument('--shading_weight', default=1e-3)
parser.add_argument('--sparsity_weight', default=1e-7)

parser.add_argument('--np_loss', default=True)
parser.add_argument('--np_weight', default=0.01)
parser.add_argument('--cn_loss', default=True)
parser.add_argument('--cn_weight', default=1.0)
parser.add_argument('--rec_loss', default=True)

### model parameters
parser.add_argument('--input_dim', type=int, default=3, help='the number of input channels')

### datasets
parser.add_argument('--norm_type', type=str, default='reconstruct')
parser.add_argument('--aug', type=str, default='') # figsc
parser.add_argument('--w', type=int, default=64, help='')
parser.add_argument('--K', type=int, default=1, help="fold")

# args = parser.parse_args(["--h", "128", "--dataset", "BUAA"])
args = parser.parse_args([])

In [3]:
# 模型导入
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
rppg_estimator = BioPhysNet(frames=args.num_rppg, device=device).to(device)
ckpt_dir = '/data/chushuyang/hemnet_exp_results/1226_1413/ckpt' 
epoch_load = 'best'
rppg_estimator.load_state_dict(torch.load(f'{ckpt_dir}/video_feature_extractor_{epoch_load}.pth', map_location=torch.device(device)))

/tmp/ipykernel_80399/322852425.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rppg_estimator.load_state_dict(torch.load(f'{ckpt_dir}/video_feature_extractor_{epoch_load

<All keys matched successfully>

In [4]:
# 数据集导入
dataloader_test = DataLoader(PURE(train=True, T=-1, norm_type=args.norm_type, aug="",w=args.w, h=args.w))
print(next(iter(dataloader_test))['ecg'].shape)

torch.Size([1, 2031])


In [ ]:
# video-level -- test
from copy import deepcopy


bpm_range = torch.arange(40, 180, dtype=torch.float).cuda()
hr_gt = []
hr_pred = []

with torch.no_grad():
    for sample_batched in tqdm(dataloader_test):
        # get the inputs
        inputs, ecg, clip_average_HR = sample_batched['video'].to(device),\
            sample_batched['ecg'].to(device), sample_batched['clip_avg_hr'].to(device)
        if "seg" in sample_batched.keys():
            mask = sample_batched['seg'].to(device)

        num_clip = 3
        input_len = inputs.shape[2]
        input_len = input_len - input_len % (num_clip * 32)
        clip_len = input_len // num_clip
        inputs = inputs[:, :, :input_len, :, :]
                        
        ecg = ecg[:, :input_len]

        new_args = deepcopy(args)
        new_args.num_rppg = clip_len
        val_rppg_estimator = BioPhysNet(frames=new_args.num_rppg, device=device).to(device)
        val_rppg_estimator.load_state_dict(torch.load(f'{ckpt_dir}/video_feature_extractor_{epoch_load}.pth', weights_only=True))
        val_rppg_estimator.eval()
        psd_gt_total = 0
        psd_pred_total = 0
        for idx in range(num_clip):

            inputs_iter = inputs[:, :, idx*clip_len : (idx+1)*clip_len, :, :]
            ecg_iter = ecg[:, idx*clip_len : (idx+1)*clip_len]
            

            psd_gt = cal_psd_hr(ecg_iter, 30, return_type='psd')
            psd_gt_total += psd_gt.view(-1).max(0)[1].cpu() + 40

            ## for rppg_estimator:
            all_inputs = {
                'video': inputs_iter,
            }
            
            if "seg" in sample_batched.keys():
                mask_clip = mask[:, idx*clip_len : (idx+1)*clip_len, :, :].to(device)
                all_inputs["seg"] = mask_clip
                
            outputs = val_rppg_estimator(all_inputs)
            rPPG = outputs['rppg']

            psd_pred = cal_psd_hr(rPPG[0], 30, return_type='psd')
            psd_pred_total += psd_pred.view(-1).max(0)[1].cpu() + 40

        hr_pred.append(psd_pred_total / num_clip)
        hr_gt.append(psd_gt_total / num_clip)


print(f'mae of model: {np.mean(np.abs(np.array(hr_gt) - np.array(hr_pred)))}')
print(f'rmse of model: {np.sqrt(np.mean(np.square(np.array(hr_gt) - np.array(hr_pred))))}')